In [1]:
count = 0

In [2]:
#load file in as a read file 
loadedjson = open('meta_Clothing_Shoes_and_Jewelry.json', 'r')

In [ ]:

allproducts = {}
listofcategories = {}
#go thorugh each line at a time (count 1, is going through one line of the data
#dividing count by 10000 with no remainder then print it)
for aline in loadedjson:
    count += 1 
    if count % 100000 == 0:
        print(count)
    aproduct = eval(aline)
    
    allproducts[aproduct['asin']] = aproduct

    for categories in aproduct['categories']:
        for acategory in categories:
            if acategory in listofcategories:
                listofcategories[acategory] += 1
            if acategory not in listofcategories:
                listofcategories[acategory] = 1

100000
200000
300000
400000


In [ ]:
#extracts asins 
count = 0 
alladidasasins = set()
for aproduct in allproducts:
    theproduct = allproducts[aproduct]
    count += 1
    if count % 100000 == 0:
        print (count/1503384)
    for categories in theproduct['categories']:
        for acategory in categories:
             if 'adidas' in acategory.lower():
                 alladidasasins.add(theproduct['asin'])

print (alladidasasins)


In [ ]:

#upload reviews data set 
loadedjson = open('reviews_Clothing_Shoes_and_Jewelry.json', 'r') 
allreviews = {}

In [ ]:
count = 0
#extract reviews
for aline in loadedjson:
    count += 1 
    if count % 100000 == 0:
        print(count)
    areview = eval(aline)
    theasin = areview['asin']
    thereviewer = areview['reviewerID']
    
    if theasin in alladidasasins:
        thekey = '%s.%s' % (theasin,thereviewer)
        allreviews[thekey] = areview


In [9]:
import json

In [10]:
# all adidas reviews into json file 
json.dump(allreviews, open('alladidasreviews.json', 'w'))
allreviews = json.load(open('alladidasreviews.json', 'r'))
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans 


ModuleNotFoundError: No module named 'nltk'

In [11]:
#implement stop words
stop_words = stopwords.words('english')
stop_words.append('adidas')

NameError: name 'stopwords' is not defined

In [12]:
#create 2 sets for good and bad reviews for the model
#good reviews
goodreviews = set()
def load_texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            #reviews that are under 2 stars
            if int(topicdata[areview]['overall']) >4:    
                reviewtext = topicdata[areview] ['reviewText']
                date = topicdata[areview]['reviewTime']
                datenocomma = date.replace(",","")
                monthdateyear = datenocomma.split(' ')     
                if monthdateyear[2] == '2013':
                    summary = topicdata[areview]['summary']
                    asin = topicdata[areview]['asin']
                    review = '%s %s %s' % (asin, summary, reviewtext)
                    goodreviews.add(review)
                    
load_texts(allreviews)
gooddocuments = list(goodreviews)

In [13]:
#create matrix with vectorizer
vectorizer = TfidfVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform(gooddocuments)

NameError: name 'TfidfVectorizer' is not defined

In [ ]:
#K Means Model 
#True k is number of topic models 
true_k = 8
#clusters is number of models 
model = KMeans (n_clusters=true_k, max_iter=100000)
#fit model. X is matrix of data 
model.fit(X)

In [ ]:
#prints topics
print('Top terms per cluster for good reviews')

In [ ]:

#looking in the model at the clusters and printing the order or the clusters based on their distances
order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

In [ ]:
for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' % (i, ' '.join(topic_terms)))


In [ ]:
#bad reviews set 
badreviews = set()
def load_texts(topicdata):
    for areview in topicdata:
        if 'reviewText' in topicdata[areview]:
            #reviews that are under 2 stars
            if int(topicdata[areview]['overall']) <2:    
                reviewtext = topicdata[areview] ['reviewText']
                date = topicdata[areview]['reviewTime']
                datenocomma = date.replace(",","")
                monthdateyear = datenocomma.split(' ')     
                if monthdateyear[2] == '2014':
                    summary = topicdata[areview]['summary']
                    asin = topicdata[areview]['asin']
                    review = '%s %s %s' % (asin, summary, reviewtext)
                    badreviews.add(review)

load_texts(allreviews)
baddocuments = list(badreviews)


In [ ]:
vectorizer = TfidfVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform(baddocuments)


In [ ]:
#topic models 
true_k = 8
# clusters is number of models 
model = KMeans (n_clusters=true_k, max_iter=100000)
#fit model. X is matrix of data 
model.fit(X)

In [ ]:
#print topics
print('Top terms per cluster for bad reviews')

In [ ]:
#looking in the model at the clusters and printing the order or the clusters based on their distances
order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()


In [ ]:
for i in range(true_k):
    topic_terms = [terms[ind] for ind in order_centroids[i,:4]]
    print('%d: %s' % (i, ' '.join(topic_terms)))

In [ ]:
import os 
outfiles = {}


In [ ]:
try:
    os.mkdir('output')
except OSError:
    print ('directory already exists')
else:
    print ('successfully created the directory')
    

In [ ]:
#creating output files to join topic terms together   
for atopic in range(true_k):
    topicterms = [terms[ind] for ind in order_centroids[atopic, :4]]
    outfiles[atopic] = open(os.path.join('output', '_'.join(topicterms)+ '.txt'), 'w')


In [ ]:
for areview in allreviews:
    if 'reviewText' in allreviews[areview]:
        thereview = allreviews[areview]
        review = '%s %s %s' % (thereview['asin'], thereview['summary'], thereview['reviewText'])
        Y = vectorizer.transform([review])
       
        for prediction in model.predict(Y):
            outfiles[prediction].write('%s\n' % review)

for n, f in outfiles.items():
    f.close()

100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000


SyntaxError: EOL while scanning string literal (<string>, line 1)